In [17]:
import pandas as pd
from loguru import logger
import json

In [18]:
df_raw = pd.read_csv('../data/bronze/telecom-customer-churn.csv')
logger.info(f"Data loaded with shape: {df_raw.shape}")
# logger.info(f"Columns: {df_raw.columns}")

2024-10-10 01:41:54.850 | INFO     | __main__:<module>:2 - Data loaded with shape: (100000, 100)


In [19]:
#load the metadata from a json file:
with open('../data/bronze/telecom-customer-metadata.json') as f:
    metadata = json.load(f)

col_desc_list = metadata['recordSet'][0]['field']
# get name and description for eac of the element in the list
col_desc_dict = {el['name']: el['description'] for el in col_desc_list}

In [20]:
# we can use the dict to see the description of the columns
col_desc_dict['months']

'Total number of months in service'

In [21]:
# rearrange the columns:
# put the label column at the end (churn)
# put the ID column at the beginning (Customer_ID)
label_col = 'churn'
id_col = 'Customer_ID'
columns = df_raw.columns.tolist()
columns.remove(label_col)
columns.remove(id_col)
columns.insert(0, id_col)
columns.append(label_col)
df_raw = df_raw[columns]

# only retain columns with no missing values
retained_cols = df_raw.columns[df_raw.isnull().mean() == 0]


In [22]:
# split to train and test:
from sklearn.model_selection import train_test_split

df_raw_train, df_raw_test = train_test_split(df_raw, test_size=0.2, random_state=99, stratify=df_raw['churn'])

In [23]:
# export the df_raw_test
df_raw_test.to_csv('../data/inference/df_raw_test.csv', index=False)

## 2. Cleaning and Featurization

The following steps are to be reproduced if requests from raw data:
- Remove cols that were removed from the initial steps
- Scaled numberical values by the saved scaler object
- One-hot encode the categorical cols

In [56]:
# remove columns with missing values:
df_train = df_raw_train[retained_cols]
labels = df_train[label_col].to_list()
ids = df_train[id_col].to_list()

num_cols = df_train.select_dtypes(include='number').columns.tolist()
#exclude the label column and the ID column
num_cols.remove(label_col)
num_cols.remove(id_col)

cat_cols = df_train.select_dtypes(exclude='number').columns.tolist()

In [36]:
df_train.shape

(80000, 57)

### Featurize the numberical cols

In [37]:
df_train_scaled = df_train[num_cols].copy()

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_train_scaled[num_cols] = scaler.fit_transform(df_train_scaled)

In [38]:
df_train_scaled.shape

(80000, 52)

### Encode the categorical cols

In [39]:
df_train_encoded = df_train[cat_cols].copy()

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = encoder.fit_transform(df_train_encoded)

columns_encoded = encoder.get_feature_names_out()
df_train_encoded = pd.DataFrame(encoded_data, columns=columns_encoded)

In [41]:
df_train_encoded.shape

(80000, 55)

### Merge to final featurized data

In [58]:
# combine the scaled and encoded data horizontally to the final dataframe
# need to reset index to avoid pandas doing outer-joining
df_train_scaled.reset_index(drop=True, inplace=True)
df_train_encoded.reset_index(drop=True, inplace=True)
df_train_featurized = pd.concat([df_train_scaled, df_train_encoded], axis=1)
# add the label and ID columns
df_train_featurized[id_col] = ids
df_train_featurized[label_col] = labels

In [59]:
df_train_featurized.shape

(80000, 109)

### Export it to Gold layer

In [60]:
df_train_featurized.to_parquet('../data/gold/telecom-customer-churn-featurized.parquet', index=False)

In [61]:
df_train_featurized

,drop_vce_Mean,drop_dat_Mean,blck_vce_Mean,blck_dat_Mean,unan_vce_Mean,unan_dat_Mean,plcd_vce_Mean,plcd_dat_Mean,recv_vce_Mean,recv_sms_Mean,...,crclscod_Z,crclscod_Z1,crclscod_Z2,crclscod_Z4,crclscod_Z5,crclscod_ZA,crclscod_ZY,asl_flag_Y,Customer_ID,churn
0,-0.255636,-0.044145,-0.287568,-0.016558,0.013389,-0.057119,-0.176420,-0.059700,-0.245900,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1099794,0
1,-0.218783,-0.044145,-0.097401,-0.016558,0.358164,-0.057119,0.405258,-0.096753,1.649606,-0.021693,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1034754,0
2,3.687658,-0.044145,0.061072,-0.016558,1.073570,-0.057119,1.994895,-0.096753,1.318463,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1046686,0
3,-0.587315,-0.044145,0.282934,0.189073,2.056176,0.580459,0.835741,0.384928,0.168980,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1054989,0
4,-0.218783,-0.044145,-0.034012,0.189073,0.108202,-0.057119,-0.008426,-0.059700,0.210848,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1036759,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,-0.034516,-0.044145,-0.097401,-0.016558,-0.098662,-0.057119,0.413657,-0.096753,-0.101263,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1005266,1
79996,0.444575,-0.044145,-0.160790,-0.016558,1.039092,-0.057119,0.598450,-0.096753,-0.211644,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1078121,0
79997,-0.329342,-0.044145,0.029377,-0.016558,-0.486533,-0.057119,-0.224718,-0.096753,-0.519949,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1022696,0
79998,3.171713,-0.044145,-0.129095,-0.016558,0.789131,-0.057119,1.129729,-0.096753,0.975902,-0.021693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1007424,0


## 3. Persisting the artifacts from data cleaning and featurization

They are important for the service later, which will apply the same logic to the raw input data (from df_raw_test)

In [30]:
import joblib

# # save the columns_without_missing to a file
with open('../services/models/retained_cols.json', 'w') as f:
    json.dump(retained_cols.tolist(), f)


joblib.dump(scaler, '../services/models/scaler.pkl')
joblib.dump(encoder, '../services/models/encoder.pkl')



['../services/models/encoder.pkl']